# Rationale Model

### Hyperparameters

In [25]:
max_sentlen, max_doclen = 5, 3
word_dim, vocab_size = 5, 10
nb_filter, filter_len = 2, 1

nb_class = 2

### Define Model

In [48]:
from keras.layers import Input, Embedding, Dense, Reshape, Permute
from keras.layers import Convolution2D, MaxPooling2D, Flatten, LSTM
from keras.models import Model

input = Input(shape=[max_sentlen*max_doclen], dtype='int32')

embedded = Embedding(output_dim=word_dim, input_dim=vocab_size, input_length=max_sentlen*max_doclen)(input)
reshaped = Reshape([1, max_sentlen, word_dim*max_doclen])(embedded)
convolved = Convolution2D(nb_filter, nb_row=filter_len, nb_col=word_dim, subsample=[1, word_dim])(reshaped)
pooled = MaxPooling2D(pool_size=[max_sentlen-filter_len+1, 1])(convolved)
permuted = Permute([2, 3, 1])(pooled)
activations = Reshape([max_doclen, nb_filter])(permuted)
hidden = LSTM(output_dim=nb_filter)(activations)
probs = Dense(output_dim=nb_class, activation='softmax')(hidden)

model = Model(input=input, output=probs)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_32 (InputLayer)            (None, 15)            0                                            
____________________________________________________________________________________________________
embedding_32 (Embedding)         (None, 15, 5)         50          input_32[0][0]                   
____________________________________________________________________________________________________
reshape_38 (Reshape)             (None, 1, 5, 15)      0           embedding_32[0][0]               
____________________________________________________________________________________________________
convolution2d_25 (Convolution2D) (None, 2, 5, 3)       12          reshape_38[0][0]                 
___________________________________________________________________________________________

### Train Model

In [17]:
from keras.utils.np_utils import to_categorical

X, y = np.array([[1, 2, 4], [0, 3, 4]]), to_categorical([1, 0])

model.fit(X, y)

Epoch 1/10
2/2 [==============================] - 0s - loss: 0.6947 - acc: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.6943 - acc: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.6939 - acc: 0.5000
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.6935 - acc: 0.5000
Epoch 5/10
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.5000
Epoch 6/10
2/2 [==============================] - 0s - loss: 0.6930 - acc: 0.5000
Epoch 7/10
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.5000
Epoch 8/10
2/2 [==============================] - 0s - loss: 0.6929 - acc: 0.5000
Epoch 9/10
2/2 [==============================] - 0s - loss: 0.6928 - acc: 0.5000
Epoch 10/10
2/2 [==============================] - 0s - loss: 0.6928 - acc: 0.5000
